This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [31]:
import pandas as pd

file_name = "./data_000637.txt"

N = 102000

data = pd.read_csv(file_name, nrows = N)
data


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
101995,1,0,124,3869201181,1278,17
101996,1,0,63,3869201181,1276,2
101997,1,1,3,3869201181,1288,19
101998,1,0,48,3869201181,1279,15


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [24]:
import numpy as np

x = np.max(data['BX_COUNTER'])
print(x)

3563


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [20]:
data = pd.read_csv(file_name)
m = data['TDC_MEAS']* 25 / 30 + data['BX_COUNTER'] * 25 + data['ORBIT_CNT'] * x * 25

print('The duration of the data taking in seconds:', (m.iloc[len(m)-1]-m.iloc[0])*10**-9, 's')
print('the duration of the data taking in minutes:', ((m.iloc[len(m)-1]-m.iloc[0])*10**-9)/60, 'min')
print('the duration of the data taking in hours:', ((m.iloc[len(m)-1]-m.iloc[0])*10**-9)/360, 'h')

The duration of the data taking in seconds: 0.9804160933125 s
the duration of the data taking in minutes: 0.016340268221875 min
the duration of the data taking in hours: 0.0027233780369791665 h


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [30]:
data['Time'] = 25/30*data['TDC_MEAS'] + data['BX_COUNTER']*25 + (m+1)*25*data['ORBIT_CNT'] 
data.head()

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,Time
0,1,0,123,3869200167,2374,26,NaN
1,1,0,124,3869200167,2374,27,NaN
2,1,0,63,3869200167,2553,28,NaN
3,1,0,64,3869200167,2558,19,NaN
4,1,0,64,3869200167,2760,25,NaN


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [29]:
new_data = data.groupby('TDC_CHANNEL').sum()

m = new_data['HEAD'].nlargest(3)

print(m, '\n')

TDC_CHANNEL
139    108059
64      66020
63      64642
Name: HEAD, dtype: int64 



6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [25]:
non_empty = data.groupby(['ORBIT_CNT']).ngroups

m = data[data['TDC_CHANNEL'] == 139].groupby(['ORBIT_CNT']).ngroups

print('Number of non-empty orbits is:', non_empty)
print('Number of unique orbits with at least one measurement form TDC_CHANNEL=139:', m)


Number of non-empty orbits is: 11001
Number of unique orbits with at least one measurement form TDC_CHANNEL=139: 10976


7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel